# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = []
    target_id_text = []
    tmp_sentence= []
    
    for sentence in source_text.split("\n"):
        sentence = " ".join(sentence.split())
        for word in sentence.split(" "):
            if (word != ''):
                tmp_sentence.append(source_vocab_to_int[word])
        source_id_text.append(tmp_sentence)
        tmp_sentence = []
    
    for sentence in target_text.split("\n"):
        sentence = " ".join(sentence.split())
        for word in sentence.split(" "):
            if (word != ''):
                tmp_sentence.append(target_vocab_to_int[word])
        if(word == "."):
            tmp_sentence.append(target_vocab_to_int["<EOS>"])
        target_id_text.append(tmp_sentence)
        tmp_sentence = []
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


C:\Users\Mario\Miniconda3\envs\dl\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    return inputs, targets, learning_rate, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for decoding
    :param target_data: Target Placeholder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    output = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [15]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    # TODO: Implement Function

    enc_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
    output, state = tf.nn.dynamic_rnn(enc_cell, rnn_inputs, dtype=tf.float32)
    return state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [46]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    # TODO: Implement Function
    train_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell, train_decoder_fn, dec_embed_input, sequence_length, scope=decoding_scope)
    
    # Apply output function
    train_logits =  output_fn(train_pred)
    return train_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [47]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn, encoder_state, dec_embeddings, start_of_sequence_id, end_of_sequence_id, 
        maximum_length, vocab_size)
    inference_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)
    return inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [52]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function

    dec_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size)] * num_layers)
    dec_drop = tf.contrib.rnn.DropoutWrapper(dec_cell, keep_prob)

    with tf.variable_scope("decoding", reuse=None) as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None, scope=decoding_scope)
    
    with tf.variable_scope("decoding", reuse=None) as decoding_scope:
        train_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob)
    
    with tf.variable_scope("decoding", reuse=True) as decoding_scope:
        infer_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                                        target_vocab_to_int['<EOS>'], sequence_length, vocab_size, decoding_scope, 
                                        output_fn, keep_prob)
    
    return train_logits, infer_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [55]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    
    encoder_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    dec_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, inf_logits = decoding_layer(dec_embed_input, dec_embeddings, encoder_state, target_vocab_size, 
                              sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)
    
    return train_logits, inf_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [67]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 13
decoding_embedding_size = 13
# Learning Rate
learning_rate = .001
# Dropout Keep Probability
keep_probability = .75

### Build the Graph
Build the graph using the neural network you implemented.

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forums to see if anyone is having the same problem.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/1077 - Train Accuracy:  0.141, Validation Accuracy:  0.179, Loss:  5.890
Epoch   0 Batch    1/1077 - Train Accuracy:  0.100, Validation Accuracy:  0.179, Loss:  5.874
Epoch   0 Batch    2/1077 - Train Accuracy:  0.056, Validation Accuracy:  0.179, Loss:  5.861
Epoch   0 Batch    3/1077 - Train Accuracy:  0.099, Validation Accuracy:  0.179, Loss:  5.844
Epoch   0 Batch    4/1077 - Train Accuracy:  0.098, Validation Accuracy:  0.179, Loss:  5.828
Epoch   0 Batch    5/1077 - Train Accuracy:  0.262, Validation Accuracy:  0.306, Loss:  5.804
Epoch   0 Batch    6/1077 - Train Accuracy:  0.245, Validation Accuracy:  0.306, Loss:  5.783
Epoch   0 Batch    7/1077 - Train Accuracy:  0.227, Validation Accuracy:  0.306, Loss:  5.758
Epoch   0 Batch    8/1077 - Train Accuracy:  0.233, Validation Accuracy:  0.306, Loss:  5.723
Epoch   0 Batch    9/1077 - Train Accuracy:  0.236, Validation Accuracy:  0.306, Loss:  5.683
Epoch   0 Batch   10/1077 - Train Accuracy:  0.199, Validati

Epoch   0 Batch   88/1077 - Train Accuracy:  0.288, Validation Accuracy:  0.342, Loss:  3.339
Epoch   0 Batch   89/1077 - Train Accuracy:  0.277, Validation Accuracy:  0.342, Loss:  3.327
Epoch   0 Batch   90/1077 - Train Accuracy:  0.275, Validation Accuracy:  0.342, Loss:  3.356
Epoch   0 Batch   91/1077 - Train Accuracy:  0.338, Validation Accuracy:  0.342, Loss:  3.149
Epoch   0 Batch   92/1077 - Train Accuracy:  0.309, Validation Accuracy:  0.342, Loss:  3.253
Epoch   0 Batch   93/1077 - Train Accuracy:  0.280, Validation Accuracy:  0.342, Loss:  3.329
Epoch   0 Batch   94/1077 - Train Accuracy:  0.297, Validation Accuracy:  0.342, Loss:  3.252
Epoch   0 Batch   95/1077 - Train Accuracy:  0.327, Validation Accuracy:  0.342, Loss:  3.184
Epoch   0 Batch   96/1077 - Train Accuracy:  0.298, Validation Accuracy:  0.342, Loss:  3.266
Epoch   0 Batch   97/1077 - Train Accuracy:  0.308, Validation Accuracy:  0.342, Loss:  3.248
Epoch   0 Batch   98/1077 - Train Accuracy:  0.333, Validati

Epoch   0 Batch  176/1077 - Train Accuracy:  0.285, Validation Accuracy:  0.363, Loss:  2.993
Epoch   0 Batch  177/1077 - Train Accuracy:  0.273, Validation Accuracy:  0.363, Loss:  3.080
Epoch   0 Batch  178/1077 - Train Accuracy:  0.323, Validation Accuracy:  0.364, Loss:  2.912
Epoch   0 Batch  179/1077 - Train Accuracy:  0.296, Validation Accuracy:  0.366, Loss:  3.015
Epoch   0 Batch  180/1077 - Train Accuracy:  0.306, Validation Accuracy:  0.366, Loss:  2.951
Epoch   0 Batch  181/1077 - Train Accuracy:  0.318, Validation Accuracy:  0.365, Loss:  2.918
Epoch   0 Batch  182/1077 - Train Accuracy:  0.333, Validation Accuracy:  0.365, Loss:  2.869
Epoch   0 Batch  183/1077 - Train Accuracy:  0.291, Validation Accuracy:  0.365, Loss:  2.999
Epoch   0 Batch  184/1077 - Train Accuracy:  0.302, Validation Accuracy:  0.366, Loss:  2.927
Epoch   0 Batch  185/1077 - Train Accuracy:  0.301, Validation Accuracy:  0.366, Loss:  2.947
Epoch   0 Batch  186/1077 - Train Accuracy:  0.299, Validati

Epoch   0 Batch  265/1077 - Train Accuracy:  0.374, Validation Accuracy:  0.418, Loss:  2.693
Epoch   0 Batch  266/1077 - Train Accuracy:  0.384, Validation Accuracy:  0.419, Loss:  2.635
Epoch   0 Batch  267/1077 - Train Accuracy:  0.391, Validation Accuracy:  0.419, Loss:  2.628
Epoch   0 Batch  268/1077 - Train Accuracy:  0.369, Validation Accuracy:  0.419, Loss:  2.723
Epoch   0 Batch  269/1077 - Train Accuracy:  0.330, Validation Accuracy:  0.419, Loss:  2.874
Epoch   0 Batch  270/1077 - Train Accuracy:  0.333, Validation Accuracy:  0.419, Loss:  2.875
Epoch   0 Batch  271/1077 - Train Accuracy:  0.373, Validation Accuracy:  0.419, Loss:  2.722
Epoch   0 Batch  272/1077 - Train Accuracy:  0.381, Validation Accuracy:  0.419, Loss:  2.658
Epoch   0 Batch  273/1077 - Train Accuracy:  0.389, Validation Accuracy:  0.419, Loss:  2.638
Epoch   0 Batch  274/1077 - Train Accuracy:  0.371, Validation Accuracy:  0.419, Loss:  2.677
Epoch   0 Batch  275/1077 - Train Accuracy:  0.398, Validati

Epoch   0 Batch  353/1077 - Train Accuracy:  0.352, Validation Accuracy:  0.441, Loss:  2.767
Epoch   0 Batch  354/1077 - Train Accuracy:  0.385, Validation Accuracy:  0.444, Loss:  2.649
Epoch   0 Batch  355/1077 - Train Accuracy:  0.400, Validation Accuracy:  0.441, Loss:  2.609
Epoch   0 Batch  356/1077 - Train Accuracy:  0.394, Validation Accuracy:  0.439, Loss:  2.596
Epoch   0 Batch  357/1077 - Train Accuracy:  0.430, Validation Accuracy:  0.435, Loss:  2.432
Epoch   0 Batch  358/1077 - Train Accuracy:  0.363, Validation Accuracy:  0.436, Loss:  2.676
Epoch   0 Batch  359/1077 - Train Accuracy:  0.388, Validation Accuracy:  0.439, Loss:  2.589
Epoch   0 Batch  360/1077 - Train Accuracy:  0.390, Validation Accuracy:  0.441, Loss:  2.615
Epoch   0 Batch  361/1077 - Train Accuracy:  0.364, Validation Accuracy:  0.441, Loss:  2.681
Epoch   0 Batch  362/1077 - Train Accuracy:  0.414, Validation Accuracy:  0.442, Loss:  2.519
Epoch   0 Batch  363/1077 - Train Accuracy:  0.377, Validati

...

Epoch  19 Batch  705/1077 - Train Accuracy:  0.916, Validation Accuracy:  0.925, Loss:  0.053
Epoch  19 Batch  706/1077 - Train Accuracy:  0.881, Validation Accuracy:  0.926, Loss:  0.074
Epoch  19 Batch  707/1077 - Train Accuracy:  0.925, Validation Accuracy:  0.925, Loss:  0.043
Epoch  19 Batch  708/1077 - Train Accuracy:  0.921, Validation Accuracy:  0.925, Loss:  0.040
Epoch  19 Batch  709/1077 - Train Accuracy:  0.935, Validation Accuracy:  0.919, Loss:  0.052
Epoch  19 Batch  710/1077 - Train Accuracy:  0.930, Validation Accuracy:  0.912, Loss:  0.032
Epoch  19 Batch  711/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.913, Loss:  0.058
Epoch  19 Batch  712/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.913, Loss:  0.041
Epoch  19 Batch  713/1077 - Train Accuracy:  0.940, Validation Accuracy:  0.921, Loss:  0.037
Epoch  19 Batch  714/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.908, Loss:  0.053
Epoch  19 Batch  715/1077 - Train Accuracy:  0.940, Validati

Epoch  19 Batch  793/1077 - Train Accuracy:  0.950, Validation Accuracy:  0.928, Loss:  0.037
Epoch  19 Batch  794/1077 - Train Accuracy:  0.924, Validation Accuracy:  0.929, Loss:  0.033
Epoch  19 Batch  795/1077 - Train Accuracy:  0.893, Validation Accuracy:  0.929, Loss:  0.049
Epoch  19 Batch  796/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.927, Loss:  0.041
Epoch  19 Batch  797/1077 - Train Accuracy:  0.957, Validation Accuracy:  0.921, Loss:  0.031
Epoch  19 Batch  798/1077 - Train Accuracy:  0.940, Validation Accuracy:  0.917, Loss:  0.036
Epoch  19 Batch  799/1077 - Train Accuracy:  0.909, Validation Accuracy:  0.912, Loss:  0.046
Epoch  19 Batch  800/1077 - Train Accuracy:  0.928, Validation Accuracy:  0.905, Loss:  0.037
Epoch  19 Batch  801/1077 - Train Accuracy:  0.956, Validation Accuracy:  0.901, Loss:  0.040
Epoch  19 Batch  802/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.904, Loss:  0.042
Epoch  19 Batch  803/1077 - Train Accuracy:  0.939, Validati

Epoch  19 Batch  881/1077 - Train Accuracy:  0.936, Validation Accuracy:  0.915, Loss:  0.040
Epoch  19 Batch  882/1077 - Train Accuracy:  0.916, Validation Accuracy:  0.916, Loss:  0.044
Epoch  19 Batch  883/1077 - Train Accuracy:  0.924, Validation Accuracy:  0.922, Loss:  0.051
Epoch  19 Batch  884/1077 - Train Accuracy:  0.918, Validation Accuracy:  0.926, Loss:  0.043
Epoch  19 Batch  885/1077 - Train Accuracy:  0.932, Validation Accuracy:  0.917, Loss:  0.030
Epoch  19 Batch  886/1077 - Train Accuracy:  0.934, Validation Accuracy:  0.920, Loss:  0.039
Epoch  19 Batch  887/1077 - Train Accuracy:  0.930, Validation Accuracy:  0.925, Loss:  0.047
Epoch  19 Batch  888/1077 - Train Accuracy:  0.923, Validation Accuracy:  0.929, Loss:  0.053
Epoch  19 Batch  889/1077 - Train Accuracy:  0.932, Validation Accuracy:  0.921, Loss:  0.043
Epoch  19 Batch  890/1077 - Train Accuracy:  0.953, Validation Accuracy:  0.921, Loss:  0.047
Epoch  19 Batch  891/1077 - Train Accuracy:  0.945, Validati

Epoch  19 Batch  969/1077 - Train Accuracy:  0.919, Validation Accuracy:  0.938, Loss:  0.048
Epoch  19 Batch  970/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.933, Loss:  0.041
Epoch  19 Batch  971/1077 - Train Accuracy:  0.949, Validation Accuracy:  0.933, Loss:  0.049
Epoch  19 Batch  972/1077 - Train Accuracy:  0.934, Validation Accuracy:  0.926, Loss:  0.036
Epoch  19 Batch  973/1077 - Train Accuracy:  0.966, Validation Accuracy:  0.919, Loss:  0.028
Epoch  19 Batch  974/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.930, Loss:  0.039
Epoch  19 Batch  975/1077 - Train Accuracy:  0.938, Validation Accuracy:  0.924, Loss:  0.039
Epoch  19 Batch  976/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.922, Loss:  0.031
Epoch  19 Batch  977/1077 - Train Accuracy:  0.945, Validation Accuracy:  0.922, Loss:  0.025
Epoch  19 Batch  978/1077 - Train Accuracy:  0.952, Validation Accuracy:  0.921, Loss:  0.044
Epoch  19 Batch  979/1077 - Train Accuracy:  0.914, Validati

Epoch  19 Batch 1057/1077 - Train Accuracy:  0.928, Validation Accuracy:  0.899, Loss:  0.050
Epoch  19 Batch 1058/1077 - Train Accuracy:  0.924, Validation Accuracy:  0.902, Loss:  0.049
Epoch  19 Batch 1059/1077 - Train Accuracy:  0.908, Validation Accuracy:  0.902, Loss:  0.054
Epoch  19 Batch 1060/1077 - Train Accuracy:  0.916, Validation Accuracy:  0.902, Loss:  0.036
Epoch  19 Batch 1061/1077 - Train Accuracy:  0.915, Validation Accuracy:  0.913, Loss:  0.046
Epoch  19 Batch 1062/1077 - Train Accuracy:  0.927, Validation Accuracy:  0.913, Loss:  0.043
Epoch  19 Batch 1063/1077 - Train Accuracy:  0.915, Validation Accuracy:  0.914, Loss:  0.045
Epoch  19 Batch 1064/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.913, Loss:  0.040
Epoch  19 Batch 1065/1077 - Train Accuracy:  0.943, Validation Accuracy:  0.924, Loss:  0.044
Epoch  19 Batch 1066/1077 - Train Accuracy:  0.948, Validation Accuracy:  0.928, Loss:  0.030
Epoch  19 Batch 1067/1077 - Train Accuracy:  0.954, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
- Convert words not in the vocabulary, to the `<UNK>` word id.

In [72]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function     
    
    word_ids = []
    for word in sentence.lower().split(" "):
        if(word in vocab_to_int):
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [73]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [36, 126, 230, 41, 173, 145, 102]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [119, 287, 251, 235, 10, 212, 1]
  French Words: ['il', 'aimait', 'le', 'gros', 'automobile', '.', '<EOS>']


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  Additionally, the translations in this data set were made by Google translate, so the translations themselves aren't particularly good.  (We apologize to the French speakers out there!) Thankfully, for this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.